# NULL処理

データフレームの操作では、NULL（欠損値）の存在がデータ分析や計算結果に影響を及ぼすことがあります。適切なNULL処理を行うことで、データの品質を保ち、信頼性の高い結果を得ることが可能です。本章では、NULL値の検出、除去、置換、および活用方法について解説します。

In [2]:
import polars as pl
import numpy as np
from helper.jupyter import row

## NULLに関する計算

Polarsでは、データの欠損を表すために、データとは別にNULL情報を管理するビットマスクを使用しています。Pythonのデータをデータフレームに変換する際、`None` は自動的にNULLに変換されます。以下の例を用いて、NULLに関する計算の基本ルールを説明します。

- ❶ 各要素に対する演算では、いずれかの要素がNULLの場合、結果もNULLになります。  
- ❷ `sum()` や `mean()` などの集約計算では、NULLが自動的に無視されます。

In [2]:
df = pl.DataFrame(dict(
    A = [1, 2, None, None],
    B = [5, None, 6, None]
))

df1 = df.select(A_plus_B=pl.col('A') + pl.col('B')) #❶
df2 = df.select( 
    A_sum=pl.col('A').sum(),  #❷
    B_mean=pl.col('B').mean()
) 
row(df, df1, df2)

shape: (4, 2) A B i64 i64 1 5 2 null null 6 null null 
 shape: (4, 1) A_plus_B i64 6 null null null 
 shape: (1, 2) A_sum B_mean i64 f64 3 5.5

## NULLを処理する関数と演算式

Polarsでは、NULLを効率的に処理するための演算式や関数を提供しています。それぞれの機能と使用例を以下に示します。

- **`drop_nulls`**: NULLが含まれる行を削除します。ただし、❶列ごとに要素数が異なる場合があるため、データフレームの構造を保つために `implode()` を使用して列をリスト形式に変換する必要があります。  
- **`fill_null`**: NULLを指定した値で埋めます。  
- **`null_count`**: 各列のNULL値の個数をカウントします。  
- **`has_nulls`**: データフレーム全体または各列にNULLが含まれているかを判定します。  
- **`is_null`**: 各要素がNULLであるかをブール値で返します。  
- **`is_not_null`**: 各要素がNULLでないかをブール値で返します。

In [18]:
df = pl.DataFrame({"A": [1, 2, None, 3], "B": [None, 3, 5, None]})

# 各列のNULLを削除し、リスト形式の列に変換
df1 = df.select(pl.all().drop_nulls().implode()) #❶

# NULLを0に埋める
df2 = df.with_columns(pl.all().fill_null(0))

# 各列のNULL値の個数を取得
df3 = df.select(pl.all().null_count())

# 各列にNULLが含まれるか判定
df4 = df.select(pl.all().has_nulls())

# 各要素がNULLか判定
df5 = df.select(pl.all().is_null())

row(df, df1, df2, df3, df4, df5)

A,B
i64,i64
1,null
2,3
null,5
3,null
A,B
list[i64],list[i64]
"[1, 2, 3]","[3, 5]"
A,B
i64,i64


`DataFrame`では、同じ機能をメソッドとして使用できます。

- **`DataFrame.drop_nulls()`**: データフレーム全体からNULLを含む行を削除します。
- **`DataFrame.fill_null()`**: 指定した値でNULLを埋めます。
- **`DataFrame.null_count()`**: 各列のNULL値の個数を取得します。

In [21]:
row(
    df.drop_nulls(),
    df.fill_null(0),
    df.null_count()
)

A,B
i64,i64
2,3
A,B
i64,i64
1,0
2,3
0,5
3,0
A,B
u32,u32


`fill_null` メソッドと計算式では、`strategy` 引数を使用してNULL値を埋める方法を指定できます。また、`limit` 引数を指定することで、NULLを埋める回数を制限することが可能です。

`strategy` には以下のオプションがあります：

- `"forward"`: 前の値でNULLを埋めます（前方補完）。
- `"backward"`: 次の値でNULLを埋めます（後方補完）。
- `"min"`: 列の最小値でNULLを埋めます。
- `"max"`: 列の最大値でNULLを埋めます。
- `"mean"`: 列の平均値でNULLを埋めます。
- `"one"`: 値をすべて1に置き換えます。
- `"zero"`: 値をすべて0に置き換えます。

`limit` は、NULL値を埋める最大回数を指定します。これにより、全てのNULLを埋めずに制限をかけることができます。

In [25]:
df = pl.DataFrame({
    "A": [1.0, None, None, 4],
    "B": [None, 2.0, None, None]
})

# 前方補完でNULLを埋める
df_forward = df.fill_null(strategy="forward")

# 後方補完でNULLを埋める
df_backward = df.fill_null(strategy="backward")

# 平均値でNULLを埋める
df_mean = df.fill_null(strategy="mean")

# 前方補完を使用し、最大1つのNULLのみ埋める
df_limit = df.fill_null(strategy="forward", limit=1)

row(df, df_forward, df_backward, df_mean, df_limit)

A,B
f64,f64
1.0,null
null,2.0
null,null
4.0,null
A,B
f64,f64
1.0,null
1.0,2.0
1.0,2.0


## NULLに関する引数

演算式には、以下のNULLに関する引数があります。

* `sort`, `sort_by`, `arg_sort`, `arg_sort_by`の`nulls_last`
* `all`, `any`, `concat_str`, `ewm_mean`, `ewm_std`, `ewm_var`の`ignore_nulls`
* `diff`の`null_behavior`
* `map_elements`の`skip_nulls`

`DataFrame`のメソッドには以下のNULLに関する引数があります。

* `DataFrame`の`nan_to_null`
* `equals`の`null_equal`
* `join`の`join_nulls`
* `mean_horizontal`, `sum_horizontal`の`ignore_nulls`
* `sort`の`nulls_last`
* `write_csv`の`null_value`
* `update`の`include_nulls`

グローバル関数のNULLに関する引数

* `read_csv`, `scan_csv`, `read_csv_batched`の`null_values`
* `from_pandas`の`nan_to_null`

## NaN処理

NaN（Not a Number）は、浮動小数点型特有の値で、計算エラーや未定義の結果を表すために使用されます。Polarsでは、NaNを効率的に処理するための方法が提供されています。以下の例を使用して、NaN処理の基本を説明します。

In [3]:
import numpy as np
import polars as pl

df = pl.DataFrame(dict(
    A=[0.0, 1.0, 2.0, None, 3.0],  # 列AにはNULLも含まれる
    B=[0.0, 1.0, 2.0, np.nan, 3.0]  # 列BにはNaNが含まれる
))

# 各列の合計を計算 (NaNやNULLは無視される)
df1 = df.select(pl.all().sum())

# NaNをNULLに置き換えてから、各列の合計を計算
df2 = df.select(pl.all().fill_nan(None).sum())

row(df, df1, df2)

A,B
f64,f64
0.0,0.0
1.0,1.0
2.0,2.0
null,NaN
3.0,3.0
A,B
f64,f64
6.0,NaN
A,B


`Inf`（Infinity）は、浮動小数点型の特殊な値で、数学的な無限を表します。PolarsにはInfを直接NULLに置き換える専用の演算式がありませんが、`replace()` を使用して処理できます。

In [9]:
df.select(
    r1=(1 / pl.col('A')).mean(),
    r2=(1 / pl.col('A')).replace(np.inf, None).mean()
)

r1,r2
f64,f64
inf,0.611111
